In [7]:
from pulp import *

dni = ['PN', 'WT', 'SR', 'CZ', 'PT']
#słownik dni tygodnia i ilości godzin
dni_tyg = {'PN' : 2, 
           'WT' : 1, 
           'SR' : 1, 
           'CZ' : 1, 
           'PT' : 3}
#pracownicy i ich stawki
pracownicy = {'Ania' : 100,
              'Stefan' : 50,
              'Hektor' : 60,
              'Olaf' : 40,
              'Lidia' : 110,
              'Piotr' : 70}
#Dostępność pracowników w tygodniu, binarnie
Ania = {
    'PN' : 1,
    'WT' : 1,
    'SR' : 1, 
    'CZ' : 1, 
    'PT' : 1
}
Stefan = {
    'PN' : 1,
    'WT' : 0,
    'SR' : 0, 
    'CZ' : 0, 
    'PT' : 0
}
Hektor = {
    'PN' : 1,
    'WT' : 1,
    'SR' : 1, 
    'CZ' : 0, 
    'PT' : 0
}
Olaf = {
    'PN' : 1,
    'WT' : 1,
    'SR' : 1, 
    'CZ' : 1, 
    'PT' : 1
}
Lidia = {
    'PN' : 0,
    'WT' : 0,
    'SR' : 0, 
    'CZ' : 1, 
    'PT' : 1
}
Piotr = {
    'PN' : 1,
    'WT' : 1,
    'SR' : 1, 
    'CZ' : 0, 
    'PT' : 0
}

In [29]:
prob = LpProblem("scheduling", LpMinimize)

#zmienne decyzyjne - jest w pracy lub nie, binarnie
Ania_var = LpVariable.dicts("Ania", dni, 0, 1, LpInteger)
Stefan_var = LpVariable.dicts("Stefan", dni, 0, 1, LpInteger)
Hektor_var = LpVariable.dicts("Hektor", dni, 0, 1, LpInteger)
Olaf_var = LpVariable.dicts("Olaf", dni, 0, 1, LpInteger)
Lidia_var = LpVariable.dicts("Lidia", dni, 0, 1, LpInteger)
Piotr_var = LpVariable.dicts("Piotr", dni, 0, 1, LpInteger)

#miesięczny koszt wypłat dla pracowników
prob += 4 * lpSum(pracownicy["Ania"] * [Ania_var[i]     for i in dni] + 
                 pracownicy["Stefan"] * [Stefan_var[i] for i in dni] + 
                 pracownicy["Hektor"] * [Hektor_var[i] for i in dni] + 
                 pracownicy["Olaf"] * [Olaf_var[i]     for i in dni] + 
                 pracownicy["Lidia"] * [Lidia_var[i]   for i in dni] + 
                 pracownicy["Piotr"] * [Piotr_var[i]   for i in dni]), "Wypłaty"

#ograniczenia co do ilości pracowników w danym dniu
for i in dni:
    prob += [Ania_var[i] +
            Stefan_var[i] + 
            Hektor_var[i] + 
            Olaf_var[i] + 
            Lidia_var[i] + 
            Piotr_var[i]] == dni_tyg[i], "Pracownicy_w_danym_dniu" 

#ograniczenia dostępności konkretnego pracownika
for i in dni:
    prob += Ania_var[i] <= Ania[i], "Workdays_Ania" + i
    prob += Stefan_var[i] <= Stefan[i], "Workdays_Stefan" + i
    prob += Hektor_var[i] <= Hektor[i], "Workdays_Hektor" + i
    prob += Olaf_var[i] <= Olaf[i], "Workdays_Olaf" + i
    prob += Lidia_var[i] <= Lidia[i], "Workdays_Lidia" + i
    prob += Piotr_var[i] <= Piotr[i], "Workdays_Piotr" + i

#ograniczneia dla pracowników, który mogą pracować więcej, niż 3 dni w tyg
prob += lpSum(Ania_var[i] for i in dni) <= 3, "workdays_Ania_limit" + i
prob += lpSum(Olaf_var[i] for i in dni) <= 3, "workdays_Olaf_limit" + i

#pokazanie wyników
prob.writeLP("scheduling1.lp")
prob.solve()
print ("Status:", LpStatus[prob.status])
for v in prob.variables():
    if(v.varValue == 1):
        print (v.name, "=", v.varValue)
    
print("Minimalny koszt = ", value(prob.objective))

Status: Optimal
Minimalny koszt =  None


In [28]:
!cat "scheduling1.lp"

\* scheduling *\
Minimize
Wypłaty: 400 Ania_CZ + 400 Ania_PN + 400 Ania_PT + 400 Ania_SR + 400 Ania_WT
 + 240 Hektor_CZ + 240 Hektor_PN + 240 Hektor_PT + 240 Hektor_SR
 + 240 Hektor_WT + 440 Lidia_CZ + 440 Lidia_PN + 440 Lidia_PT + 440 Lidia_SR
 + 440 Lidia_WT + 160 Olaf_CZ + 160 Olaf_PN + 160 Olaf_PT + 160 Olaf_SR
 + 160 Olaf_WT + 280 Piotr_CZ + 280 Piotr_PN + 280 Piotr_PT + 280 Piotr_SR
 + 280 Piotr_WT + 200 Stefan_CZ + 200 Stefan_PN + 200 Stefan_PT
 + 200 Stefan_SR + 200 Stefan_WT
Subject To
Binaries
Ania_CZ
Ania_PN
Ania_PT
Ania_SR
Ania_WT
Hektor_CZ
Hektor_PN
Hektor_PT
Hektor_SR
Hektor_WT
Lidia_CZ
Lidia_PN
Lidia_PT
Lidia_SR
Lidia_WT
Olaf_CZ
Olaf_PN
Olaf_PT
Olaf_SR
Olaf_WT
Piotr_CZ
Piotr_PN
Piotr_PT
Piotr_SR
Piotr_WT
Stefan_CZ
Stefan_PN
Stefan_PT
Stefan_SR
Stefan_WT
End
